In [ ]:
import numpy as np

In [ ]:
def gen_C2(k, gamma):
    stop = False
    while not stop:
        C = np.random.uniform(0,1,size=(k,k))
        np.fill_diagonal(C, 0)
        C = (1-gamma) * C / np.sum(C, axis=0)[np.newaxis, :]
        np.fill_diagonal(C, gamma)
        c_rank = np.linalg.matrix_rank(C)
        if c_rank == k:
            stop = True

    print("C: ", C)
    assert(np.allclose(np.sum(C, axis=0), 1))
    print("det(C): ", np.linalg.det(C))
    u, s, vh = np.linalg.svd(C)
    print("smallest singular value of C: ", s[-1])
    return C

In [ ]:
noise_rates = [0.1, 0.2, 0.3, 0.4]

In [ ]:
for d in ["vehicle", "pageblocks", "satimage", "covtype", "abalone"]:
    print(d)
    data_dict = np.load("./" + d + "/" + d +"_data.npy", allow_pickle=True).item()
    #print(data_dict.keys())
    X_data = data_dict['X']
    Y_data = data_dict['Y']


    n_class = len(np.unique(Y_data))
    print("n_class: ", n_class)

    # Generate C and noisy labels
    for i in range(len(noise_rates)):
        C = gen_C2(n_class, 1-noise_rates[i])
        np.save("./" + d + "/C_" + str(i) + ".npy", C, allow_pickle=True)

        Y_noisy = Y_data.copy()

        for j in range(len(Y_data)):
            Y_noisy[j] = np.random.choice(n_class, size=1, p=np.squeeze(C[:,Y_data[j]]))

        assert(n_class == len(np.unique(Y_noisy)))

        new_data_dict = {'X': X_data, 'Y': Y_noisy}
        np.save("./" + d + "/" + d + "_noisy_" + str(4+i) + ".npy", new_data_dict, allow_pickle=True)